# Naïve Behavioral Cloning.

### First assignment of CS(1+1)(3^2)(2^2)-112 by Sergey Levine. 
This is just a notebook and You'll need to get rest of files from course github page.

## Import Some of dependencies!

In [1]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy

/home/taesiri/anaconda3/envs/Deep3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [349]:
expert_policy_file = 'experts/Humanoid-v1.pkl'
envname = 'Humanoid-v1'
policy_fn = load_policy.load_policy(expert_policy_file)
num_rollouts = 1000

obs (1, 376) (1, 376)


## Gathering Data from expret - Unfortunately we have to use Tensorflow, Which i don't like!
You should read comments, before the cell bellow!

In [351]:
# set to true if you want to gather data from expert, or you can load data from a pickle file (see cell bellow)
gather_data = False

# only allocate a fraction of GPU memory to TF, cause it doesn't give it back!
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)


if gather_data:
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)):
        tf_util.initialize()

        env = gym.make(envname)
        max_steps = env.spec.timestep_limit

        returns = []
        observations = []
        actions = []
        for i in range(num_rollouts):
            print('iter', i)
            obs = env.reset()
            done = False
            totalr = 0.
            steps = 0
            while not done:
                action = policy_fn(obs[None,:])
                observations.append(obs)
                actions.append(action)
                obs, r, done, _ = env.step(action)
                totalr += r
                steps += 1
                # Don't Render anything!
    #             env.render()
                if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
                if steps >= max_steps:
                    break
            returns.append(totalr)

        print('returns', returns)
        print('mean return', np.mean(returns))
        print('std of return', np.std(returns))

        expert_data = {'observations': np.array(observations),
                       'actions': np.array(actions)}

[2017-12-11 13:33:02,725] Making new env: Humanoid-v1


iter 0
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 1
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 2
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 3
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 4
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 5
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 6
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 7
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 8
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 9
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
iter 10
100/1000
200

In [359]:
print(expert_data['actions'].shape)

(999094, 1, 17)


## Delete the Tensorflow!

Go home Tensorflow! You are not needed anymore

In [353]:
# I Don't Know!!!
# Stupid TF doesn't free up the memory!!!

## Save / Load Expert from file!

In [360]:
import pickle

pickle_out = open('expert-' + envname +  '.pickle', "wb")
pickle.dump(expert_data, pickle_out)
pickle_out.close()

In [8]:
import pickle

pickle_in = open('expert-' + envname +  '.pickle', "rb")
expert_data = pickle.load(pickle_in)

## Creating a Feed Forward Neural Network for Behavioral cloning 

#### YES I'm Using PyTorch

In [362]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import Dataset

import torchvision.datasets as dset
import torchvision.transforms as T

import timeit

In [363]:
gpu_dtype = torch.cuda.FloatTensor # the CPU datatype

# Constant to control how frequently we print train loss
print_every = 100

# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

## Creating a Train, Test, Validation Dataset

In [364]:
class ObservastionActionDataset(Dataset):
    """Observations Actions dataset."""

    def __init__(self, list_observations, list_actions, transform=None):
        """
        Args:
            list_observations (observation tensor): .
            list_actions (action tensor): .
        """
        self.actions = list_actions
        self.observations = list_actions
        self.transform = transform

    def __len__(self):
        return actions.shape[0]

    def __getitem__(self, idx):
        return observations[idx], actions[idx]

In [365]:
## Copy paste from CS231n

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

In [366]:
expert_observations = expert_data['observations']
expert_actions = expert_data['actions']

In [367]:
# Good old way to shuffle
# I'm not exaclty sure if I need this or pytorch Dataset will take care of this step!
import random

idx = np.arange(expert_observations.shape[0])
np.random.shuffle(idx)

expert_observations = expert_observations[idx]
expert_actions = expert_actions[idx]

In [375]:
# Tensorify!
NUM_TRAIN = 900000
NUM_VAL = 99094

train_observations = expert_observations[:NUM_TRAIN]
train_actions = expert_actions[:NUM_TRAIN, :NUM_TRAIN]
train_actions = np.squeeze(train_actions, axis=1)
train_observations = torch.from_numpy(train_observations)
train_actions = torch.from_numpy(train_actions)

test_observations = expert_observations[NUM_TRAIN:]
test_actions =  expert_actions[NUM_TRAIN:]
test_actions = np.squeeze(test_actions, axis=1)
test_observations = torch.from_numpy(test_observations)
test_actions = torch.from_numpy(test_actions)

In [376]:
train_dataset = ObservastionActionDataset(train_observations, train_actions)
test_dataset = ObservastionActionDataset(test_observations, test_actions)

In [377]:
loader_train = DataLoader(train_dataset, batch_size=64, sampler=ChunkSampler(NUM_TRAIN-NUM_VAL, 0))
loader_val = DataLoader(train_dataset, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, NUM_TRAIN-NUM_VAL))
loader_test = DataLoader(test_dataset, batch_size=64, sampler=ChunkSampler(NUM_TRAIN-NUM_VAL, 0))

## Trainner  code (Copied from CS231n Assignment 2)

In [378]:
## Copy paste from CS231n

def train(model, loss_fn, optimizer, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.type(gpu_dtype))
            # I'm not sure why we have extra dim here!
            y = torch.squeeze(y, 1)
            y_var = Variable(y.type(gpu_dtype))

            scores = model(x_var)
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            
def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(gpu_dtype), volatile=True)

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

## Creating a Simple Feed forward neural network to Fit(Observation, Action)

In [379]:
model_def = nn.Sequential( 
    nn.Linear(376, 2048),
    nn.ReLU(inplace=True),
    nn.Linear(2048, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, 17)
)

model = model_def.type(gpu_dtype)
loss_fn = nn.MSELoss().type(gpu_dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train(model, loss_fn, optimizer, num_epochs=2)
# check_accuracy(model, loader_val)

Starting epoch 1 / 2
t = 100, loss = 0.1973
t = 200, loss = 0.1784
t = 300, loss = 0.1628
t = 400, loss = 0.1296
t = 500, loss = 0.0722
t = 600, loss = 0.1590
t = 700, loss = 0.0877
t = 800, loss = 0.2120
t = 900, loss = 0.1187
t = 1000, loss = 0.1058
t = 1100, loss = 0.0700
t = 1200, loss = 0.0650
t = 1300, loss = 0.0857
t = 1400, loss = 0.1119
t = 1500, loss = 0.0609
t = 1600, loss = 0.0813
t = 1700, loss = 0.0660
t = 1800, loss = 0.0703
t = 1900, loss = 0.0634
t = 2000, loss = 0.0569
t = 2100, loss = 0.0599
t = 2200, loss = 0.0469
t = 2300, loss = 0.0965
t = 2400, loss = 0.0543
t = 2500, loss = 0.0891
t = 2600, loss = 0.0510
t = 2700, loss = 0.0719
t = 2800, loss = 0.0745
t = 2900, loss = 0.0535
t = 3000, loss = 0.0666
t = 3100, loss = 0.0550
t = 3200, loss = 0.0589
t = 3300, loss = 0.2383
t = 3400, loss = 0.0496
t = 3500, loss = 0.0411
t = 3600, loss = 0.0537
t = 3700, loss = 0.0536
t = 3800, loss = 0.0708
t = 3900, loss = 0.0374
t = 4000, loss = 0.0567
t = 4100, loss = 0.0423
t = 

## The moment of truth! Let's test what we've done!

In [399]:
env = gym.make(envname)
max_steps = env.spec.timestep_limit

model.eval() # Put the model in test mode
rewards = []

for i in range(5):
    print('iter', i)
    obs = env.reset()
    done = False
    totalr = 0.
    steps = 0
    while not done:
        x = torch.from_numpy(obs[None,:])
        x_var = Variable(x.type(gpu_dtype))
        act = model(x_var)
        action = model(x_var)
        obs, r, done, _ = env.step(action)
        totalr += r
        steps += 1
        # lemme see!
        env.render()
        if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
        if steps >= max_steps:
            break
    rewards.append(totalr)

print('returns', rewards)
print('mean return', np.mean(rewards))
print('std of return', np.std(rewards))

[2017-12-11 14:52:11,242] Making new env: Humanoid-v1


iter 0
100/1000
200/1000
iter 1
100/1000
iter 2
iter 3
100/1000
200/1000
300/1000
iter 4
100/1000
returns [1645.4591010673578, 1449.8547357677344, 615.89515231094788, 2593.2463206243037, 826.62586275556998]
mean return 1426.21623451
std of return 696.66553027


# Let's Record a video and share it with the world

In [401]:
env = gym.make(envname)
max_steps = env.spec.timestep_limit

model.eval() # Put the model in test mode
rewards = []

# Importing VideoRecorder
from gym.monitoring import VideoRecorder

for i in range(5):
    print('iter', i)
    obs = env.reset()
    done = False
    totalr = 0.
    steps = 0
    
    rec = VideoRecorder(env, path=envname + '-bc-iter' + str(i) + '.mp4')
    rec.capture_frame()
    while not done:
        x = torch.from_numpy(obs[None,:])
        x_var = Variable(x.type(gpu_dtype))
        act = model(x_var)
        action = model(x_var)
        obs, r, done, _ = env.step(action)
        totalr += r
        steps += 1
        # lemme see!
        env.render()
        rec.capture_frame()
        if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
        if steps >= max_steps:
            break
    rewards.append(totalr)
    
    rec.close()

print('returns', rewards)
print('mean return', np.mean(rewards))
print('std of return', np.std(rewards))

[2017-12-11 14:55:11,288] Making new env: Humanoid-v1
[2017-12-11 14:55:11,305] Starting new video recorder writing to Humanoid-v1-bc-iter0.mp4


iter 0
100/1000
200/1000


[2017-12-11 14:55:24,855] Starting new video recorder writing to Humanoid-v1-bc-iter1.mp4


iter 1
100/1000


[2017-12-11 14:55:36,183] Starting new video recorder writing to Humanoid-v1-bc-iter2.mp4


iter 2
100/1000
200/1000


[2017-12-11 14:55:52,590] Starting new video recorder writing to Humanoid-v1-bc-iter3.mp4


iter 3
100/1000


[2017-12-11 14:56:00,621] Starting new video recorder writing to Humanoid-v1-bc-iter4.mp4


iter 4
100/1000
200/1000
returns [1689.6606018844225, 1150.8415185131958, 2133.3740731238563, 722.62866122418541, 1907.0495294364659]
mean return 1520.71087684
std of return 515.14974935


## Lemme see

In [405]:
# Code to show Video on Jupter - Copy paste from https://stackoverflow.com/a/18026076
import io
import base64
from IPython.display import HTML

video = io.open('Humanoid-v1-bc-iter3.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

## Share the above video and tell everyone that you'd successfully cloned a humanoid and It'll kill every single one of us! :P

# Using DAgger

In [373]:
# TODO